In [ ]:
curl -fsSL https://ollama.com/install.sh | sh
nohup ollama serve &
ollama pull llama3:8b
ollama list

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
!pip install ollama duckduckgo-search beautifulsoup4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.8 MB/s eta 0:00:00


In [ ]:
ollama run llama3:8b

In [3]:
!nohup ollama serve &
!ollama pull llama3:8b

nohup: appending output to 'nohup.out'



In [6]:
!ollama list

NAME         ID              SIZE      MODIFIED           
llama3:8b    365c0bd3c000    4.7 GB    About a minute ago    


In [7]:
import os
import json
from ollama import Client
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# إعداد عميل Ollama
ollama_client = Client(host='http://localhost:11434')

# دالة البحث في الإنترنت
def search_internet(query):
    print(f"🔍 البحث في الإنترنت عن: {query}")
    ddgs = DDGS()
    results = ddgs.text(query, max_results=3)
    return results

# دالة البحث في الكمبيوتر المحلي
def search_local(query):
    print(f"💻 البحث في الكمبيوتر المحلي عن: {query}")
    search_dir = os.path.expanduser("~")  # تبدأ البحث من مجلد المستخدم الرئيسي
    results = []

    for root, dirs, files in os.walk(search_dir):
        for file in files:
            if query.lower() in file.lower():
                results.append(os.path.join(root, file))
        if len(results) >= 5:  # حد أقصى للنتائج
            break
    return results

# دالة استخراج النص من صفحة ويب
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()[:500]  # أول 500 حرف
    except:
        return "فشل استخراج المحتوى"

# دالة تحديد نوع البحث باستخدام Ollama
def decide_search_type(query):
    prompt = f"""
    أنت وكيل ذكي. بناءً على الاستفسار التالي، حدد ما إذا كان البحث يجب أن يكون:
    - "local" (في الكمبيوتر المحلي، مثل البحث عن ملفات أو مستندات)
    - "internet" (في الإنترنت، مثل البحث عن معلومات عامة أو أخبار)
    - "none" (لا يتطلب بحثًا)

    الاستفسار: "{query}"
    أجب بكلمة واحدة فقط: local أو internet أو none.
    """
    response = ollama_client.generate(model='llama3:8b', prompt=prompt)
    return response['response'].strip().lower()

# دالة تنفيذ البحث بناءً على النوع
def execute_search(query):
    search_type = decide_search_type(query)

    if search_type == "local":
        results = search_local(query)
        if results:
            return f"📂 تم العثور على الملفات التالية:\n" + "\n".join(results)
        else:
            return "❌ لم يتم العثور على ملفات مطابقة."

    elif search_type == "internet":
        search_results = search_internet(query)
        if search_results:
            response = "🌐 نتائج البحث على الإنترنت:\n"
            for result in search_results:
                title = result.get('title', 'بدون عنوان')
                link = result.get('href', '#')
                snippet = result.get('body', 'لا يوجد وصف')
                response += f"\n- {title}\n  {link}\n  {snippet}\n"
            return response
        else:
            return "❌ لم يتم العثور على نتائج على الإنترنت."

    else:
        return "ℹ️ الاستفسار لا يتطلب بحثًا."

# واجهة المستخدم الرئيسية
if __name__ == "__main__":
    print("🤖 وكيل Ollama متعدد الأدوات")
    print("اكتب استفسارك أو اكتب 'خروج' للإنهاء.")

    while True:
        user_input = input("\n> ")
        if user_input.lower() in ['خروج', 'exit', 'quit']:
            break

        response = execute_search(user_input)
        print(response)

🤖 وكيل Ollama متعدد الأدوات
اكتب استفسارك أو اكتب 'خروج' للإنهاء.

> What's the latest news on artificial intelligence?


/tmp/ipython-input-1881140069.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS()


🔍 البحث في الإنترنت عن: What's the latest news on artificial intelligence?
🌐 نتائج البحث على الإنترنت:

- Nintendo Switch 2 Consoles - Best Buy
  https://www.bestbuy.com/site/nintendo-switch-2/nintendo-switch-2-consoles/pcmcat1743185999078.c?id=pcmcat1743185999078
  Shop at Best Buy for your Nintendo Switch 2. Enjoy the Nintendo Switch 2's larger screen and doubled pixel count compared to the Nintendo …

- Best Buy Store Locator: Store Hours, Directions & Events
  https://www.bestbuy.com/site/store-locator
  Use the Best Buy store locator to find stores in your area. Then, visit each Best Buy store's page to see store hours, directions, news, events …

- Apple iPad Deals - Best Buy
  https://www.bestbuy.com/site/all-apple-on-sale/apple-ipad-on-sale/pcmcat1720705903650.c?id=pcmcat1720705903650
  Shop for Apple iPad on sale at Best Buy today. Find top deals, sales and special offers on popular Apple iPad models.


> ابحث عن ملفات csv في جهازي
💻 البحث في الكمبيوتر المحلي عن: ابحث عن ملفات

KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import json
from ollama import Client
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# إعداد عميل Ollama
ollama_client = Client(host='http://localhost:11434')

# دالة البحث في الإنترنت
def search_internet(query):
    print(f"🔍 البحث في الإنترنت عن: {query}")
    ddgs = DDGS()
    results = ddgs.text(query, max_results=3)
    return results

# دالة البحث في الكمبيوتر المحلي
def search_local(query):
    print(f"💻 البحث في الكمبيوتر المحلي عن: {query}")
    search_dir = os.path.expanduser("~")  # تبدأ البحث من مجلد المستخدم الرئيسي
    results = []

    for root, dirs, files in os.walk(search_dir):
        for file in files:
            if query.lower() in file.lower():
                results.append(os.path.join(root, file))
        if len(results) >= 5:  # حد أقصى للنتائج
            break
    return results

# دالة استخراج النص من صفحة ويب
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()[:500]  # أول 500 حرف
    except:
        return "فشل استخراج المحتوى"

# دالة تحديد نوع البحث باستخدام Ollama
def decide_search_type(query):
    prompt = f"""
    You are an intelligent agent. Based on the following query, determine whether the search should be:
    - "local" (on the local computer, such as searching for files or documents)
    - "internet" (on the Internet, such as searching for general information or news)
    - "none" (does not require a search)

    الاستفسار: "{query}"
    أجب بكلمة واحدة فقط: local أو internet أو none.
    """
    response = ollama_client.generate(model='llama3:8b', prompt=prompt)
    return response['response'].strip().lower()

# دالة تنفيذ البحث بناءً على النوع
def execute_search(query):
    search_type = decide_search_type(query)

    if search_type == "local":
        results = search_local(query)
        if results:
            return f"📂 تم العثور على الملفات التالية:\n" + "\n".join(results)
        else:
            return "❌ لم يتم العثور على ملفات مطابقة."

    elif search_type == "internet":
        search_results = search_internet(query)
        if search_results:
            response = "🌐 نتائج البحث على الإنترنت:\n"
            for result in search_results:
                title = result.get('title', 'بدون عنوان')
                link = result.get('href', '#')
                snippet = result.get('body', 'لا يوجد وصف')
                response += f"\n- {title}\n  {link}\n  {snippet}\n"
            return response
        else:
            return "❌ لم يتم العثور على نتائج على الإنترنت."

    else:
        return "ℹ️ الاستفسار لا يتطلب بحثًا."

# واجهة المستخدم الرئيسية
if __name__ == "__main__":
    print("🤖 وكيل Ollama متعدد الأدوات")
    print("اكتب استفسارك أو اكتب 'خروج' للإنهاء.")

    while True:
        user_input = input("\n> ")
        if user_input.lower() in ['خروج', 'exit', 'quit']:
            break

        response = execute_search(user_input)
        print(response)

In [ ]:
import os
import json
from ollama import Client
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# إعداد عميل Ollama
ollama_client = Client(host='http://localhost:11434')

# دالة البحث في الإنترنت
def search_internet(query):
    print(f"🔍 البحث في الإنترنت عن: {query}")
    ddgs = DDGS()
    results = ddgs.text(query + " latest news", max_results=3)
    return results

# دالة البحث في الكمبيوتر المحلي
def search_local(query):
    print(f"💻 البحث في الكمبيوتر المحلي عن: {query}")

    # إذا كان الاستفسار يحتوي على مسار مطلق مثل /content/sample_data
    if query.startswith("/"):
        search_dir = query
    else:
        search_dir = "/content"  # المسار الافتراضي في Colab

    results = []

    if os.path.exists(search_dir):
        for root, dirs, files in os.walk(search_dir):
            for file in files:
                if query.lower() in file.lower():
                    results.append(os.path.join(root, file))
            if len(results) >= 5:  # حد أقصى للنتائج
                break
    else:
        return f"❌ المسار '{search_dir}' غير موجود."

    if results:
        return f"📂 تم العثور على الملفات التالية:\n" + "\n".join(results)
    else:
        return "❌ لم يتم العثور على ملفات مطابقة."

# دالة استخراج النص من صفحة ويب
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()[:500]  # أول 500 حرف
    except:
        return "فشل استخراج المحتوى"

# دالة تحديد نوع البحث باستخدام Ollama
def decide_search_type(query):
    prompt = f"""
    أنت وكيل ذكي. بناءً على الاستفسار التالي، حدد ما إذا كان البحث يجب أن يكون:
    - "local" (في الكمبيوتر المحلي، مثل البحث عن ملفات أو مستندات)
    - "internet" (في الإنترنت، مثل البحث عن معلومات عامة أو أخبار)
    - "none" (لا يتطلب بحثًا)

    الاستفسار: "{query}"
    أجب بكلمة واحدة فقط: local أو internet أو none.
    """
    response = ollama_client.generate(model='llama3', prompt=prompt)
    return response['response'].strip().lower()

# دالة تنفيذ البحث بناءً على النوع
def execute_search(query):
    search_type = decide_search_type(query)

    if search_type == "local":
        return search_local(query)

    elif search_type == "internet":
        search_results = search_internet(query)
        if search_results:
            response = "🌐 نتائج البحث على الإنترنت:\n"
            for result in search_results:
                title = result.get('title', 'بدون عنوان')
                link = result.get('href', '#')
                snippet = result.get('body', 'لا يوجد وصف')
                response += f"\n- {title}\n  {link}\n  {snippet}\n"
            return response
        else:
            return "❌ لم يتم العثور على نتائج على الإنترنت."

    else:
        return "ℹ️ الاستفسار لا يتطلب بحثًا."

# واجهة المستخدم الرئيسية
if __name__ == "__main__":
    print("🤖 وكيل Ollama متعدد الأدوات")
    print("اكتب استفسارك أو اكتب 'خروج' للإنهاء.")

    while True:
        user_input = input("\n> ")
        if user_input.lower() in ['خروج', 'exit', 'quit']:
            break

        response = execute_search(user_input)
        print(response)

In [1]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [2]:
import os
import json
from ollama import Client
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# Setup Ollama client
ollama_client = Client(host='http://localhost:11434')

# Function to search the internet
def search_internet(query):
    print(f"🔍 Searching the internet for: {query}")
    ddgs = DDGS()
    results = ddgs.text(query + " latest news", max_results=3)
    return results

# Function to search the local computer
def search_local(query):
    print(f"💻 Searching locally for: {query}")

    # If the query contains an absolute path like /content/sample_data
    if query.startswith("/"):
        search_dir = query
    else:
        search_dir = "/content"  # Default path in Colab

    results = []

    if os.path.exists(search_dir):
        for root, dirs, files in os.walk(search_dir):
            for file in files:
                if query.lower() in file.lower():
                    results.append(os.path.join(root, file))
            if len(results) >= 5:  # Max results limit
                break
    else:
        return f"❌ Path '{search_dir}' does not exist."

    if results:
        return f"📂 Found the following files:\n" + "\n".join(results)
    else:
        return "❌ No matching files found."

# Function to extract text from a webpage
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()[:500]  # First 500 characters
    except:
        return "Failed to extract content"

# Function to decide search type using Ollama
def decide_search_type(query):
    prompt = f"""
    You are an intelligent agent. Based on the following query, determine if the search should be:
    - "local" (on the local computer, like searching for files or documents)
    - "internet" (on the internet, like searching for general information or news)
    - "none" (no search needed)

    Query: "{query}"
    Answer with only one word: local, internet, or none.
    """
    response = ollama_client.generate(model='llama3:8b]', prompt=prompt)
    return response['response'].strip().lower()

# Function to execute search based on type
def execute_search(query):
    search_type = decide_search_type(query)

    if search_type == "local":
        return search_local(query)

    elif search_type == "internet":
        search_results = search_internet(query)
        if search_results:
            response = "🌐 Internet search results:\n"
            for result in search_results:
                title = result.get('title', 'No title')
                link = result.get('href', '#')
                snippet = result.get('body', 'No description')
                response += f"\n- {title}\n  {link}\n  {snippet}\n"
            return response
        else:
            return "❌ No results found on the internet."

    else:
        return "ℹ️ The query does not require a search."

# Main user interface
if __name__ == "__main__":
    print("🤖 Ollama Multi-Tool Agent")
    print("Enter your query or type 'exit' to quit.")

    while True:
        user_input = input("\n> ")
        if user_input.lower() in ['exit', 'quit']:
            break

        response = execute_search(user_input)
        print(response)

🤖 Ollama Multi-Tool Agent
Enter your query or type 'exit' to quit.

> Search for csv files in harddesk
💻 Searching locally for: Search for csv files in harddesk
❌ No matching files found.

> Search for csv files
💻 Searching locally for: Search for csv files
❌ No matching files found.

> Search for csv files in /content/sample_data
💻 Searching locally for: Search for csv files in /content/sample_data
❌ No matching files found.

> search for files in /content/sample_data
💻 Searching locally for: search for files in /content/sample_data
❌ No matching files found.


KeyboardInterrupt: Interrupted by user

In [4]:
!nohup ollama serve &
import os
import json
from ollama import Client
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup

# Setup Ollama client
ollama_client = Client(host='http://localhost:11434')

# Function to search the internet
def search_internet(query):
    print(f"🔍 Searching the internet for: {query}")
    ddgs = DDGS()
    results = ddgs.text(query + " latest news", max_results=3)
    return results

# Function to search the local computer
def search_local(query):
    print(f"💻 Searching locally for: {query}")

    # If the query contains an absolute path like /content/sample_data
    if query.startswith("/"):
        search_dir = query
    else:
        search_dir = "/content"  # Default path in Colab

    results = []

    if os.path.exists(search_dir):
        for root, dirs, files in os.walk(search_dir):
            for file in files:
                # Check if the query mentions a file extension like 'csv'
                if 'csv' in query.lower() and file.endswith('.csv'):
                    results.append(os.path.join(root, file))
                elif query.lower() in file.lower():
                    results.append(os.path.join(root, file))
            if len(results) >= 5:  # Max results limit
                break
    else:
        return f"❌ Path '{search_dir}' does not exist."

    if results:
        return f"📂 Found the following files:\n" + "\n".join(results)
    else:
        return "❌ No matching files found."

# Function to extract text from a webpage
def extract_text_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()[:500]  # First 500 characters
    except:
        return "Failed to extract content"

# Function to decide search type using Ollama
def decide_search_type(query):
    prompt = f"""
    You are an intelligent agent. Based on the following query, determine if the search should be:
    - "local" (on the local computer, like searching for files or documents)
    - "internet" (on the internet, like searching for general information or news)
    - "none" (no search needed)

    Query: "{query}"
    Answer with only one word: local, internet, or none.
    """
    response = ollama_client.generate(model='llama3:8b', prompt=prompt)
    return response['response'].strip().lower()

# Function to execute search based on type
def execute_search(query):
    search_type = decide_search_type(query)

    if search_type == "local":
        return search_local(query)

    elif search_type == "internet":
        search_results = search_internet(query)
        if search_results:
            response = "🌐 Internet search results:\n"
            for result in search_results:
                title = result.get('title', 'No title')
                link = result.get('href', '#')
                snippet = result.get('body', 'No description')
                response += f"\n- {title}\n  {link}\n  {snippet}\n"
            return response
        else:
            return "❌ No results found on the internet."

    else:
        return "ℹ️ The query does not require a search."

# Main user interface
if __name__ == "__main__":
    print("🤖 Ollama Multi-Tool Agent")
    print("Enter your query or type 'exit' to quit.")

    while True:
        user_input = input("\n> ")
        if user_input.lower() in ['exit', 'quit']:
            break

        response = execute_search(user_input)
        print(response)

nohup: appending output to 'nohup.out'
🤖 Ollama Multi-Tool Agent
Enter your query or type 'exit' to quit.

> Search for csv files
💻 Searching locally for: Search for csv files
📂 Found the following files:
/content/sample_data/california_housing_test.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/mnist_test.csv
/content/sample_data/mnist_train_small.csv

> What's the latest news on artificial intelligence?
🔍 Searching the internet for: What's the latest news on artificial intelligence?


/tmp/ipython-input-1625273880.py:15: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS()


🌐 Internet search results:

- Nintendo Switch 2 Consoles - Best Buy
  https://www.bestbuy.com/site/nintendo-switch-2/nintendo-switch-2-consoles/pcmcat1743185999078.c?id=pcmcat1743185999078
  Shop at Best Buy for your Nintendo Switch 2. Enjoy the Nintendo Switch 2's larger screen and doubled pixel count compared to the Nintendo Switch 1. Enjoy smooth gameplay anytime, …

- Best Buy Store Locator: Store Hours, Directions & Events
  https://www.bestbuy.com/site/store-locator
  Use the Best Buy store locator to find stores in your area. Then, visit each Best Buy store's page to see store hours, directions, news, events and more.

- Apple iPad Deals - Best Buy
  https://www.bestbuy.com/site/all-apple-on-sale/apple-ipad-on-sale/pcmcat1720705903650.c?id=pcmcat1720705903650
  Shop for Apple iPad on sale at Best Buy today. Find top deals, sales and special offers on popular Apple iPad models.


> exit


🛠️ تحسينات مقترحة مستقبلًا
إذا أردت، يمكننا تحسين الوكيل أكثر، مثل:

تحسين جودة البحث على الإنترنت باستخدام أدوات أكثر تخصصًا أو تعديل الاستعلامات.
إضافة المزيد من الأدوات مثل البحث في قواعد البيانات أو قراءة محتوى الملفات.
إضافة واجهة رسومية (GUI) لسهولة الاستخدام.

https://chat.z.ai/c/aba22008-bd6c-4366-9b06-8b0fe6584739